In [2]:
import pandas as pd
from scipy.stats import chi2_contingency
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import numpy as np
from pandas.plotting import scatter_matrix


from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import BaggingClassifier, GradientBoostingClassifier, RandomForestClassifier
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier 
from sklearn.svm import SVC
from sklearn.decomposition import PCA

In [3]:
df = pd.read_excel('subastas_output_updated_v.xlsx')
df.head()

,ANUNCIO_BOE,Antigüedad,CANTIDAD_RECLAMADA,CARGAS,CODIGO,CODIGO_POSTAL,CORREO_ELECTRONICO,CUOTA,Clase,Clase de cultivo,...,TELEFONO,TIPO_DE_SUBASTA,TITULO_JURIDICO,TRAMOS_ENTRE_PUJAS,URL,Uso,VALOR_DE_TASACION,VALOR_SUBASTA,VISITABLE,VIVIENDA_HABITUAL
0,BOE-B-2019-35282,NaN,0.0,No constan,28079060HM,NaN,administracionbienes.orga@mjusticia.es,NaN,NaN,NaN,...,913904912,JUDICIAL VOLUNTARIA,NaN,50.0,https://subastas.boe.es/./detalleSubasta.php?i...,NaN,2500.0,2500.0,Si (Las visitas se podran realizar de lunes a ...,NaN
1,BOE-B-2019-35282,NaN,0.0,No constan,28079060HM,NaN,administracionbienes.orga@mjusticia.es,NaN,NaN,NaN,...,913904912,JUDICIAL VOLUNTARIA,NaN,50.0,https://subastas.boe.es/./detalleSubasta.php?i...,NaN,2500.0,2500.0,Si (Las visitas se podran realizar de lunes a ...,NaN
2,BOE-B-2019-35282,NaN,0.0,No constan,28079060HM,NaN,administracionbienes.orga@mjusticia.es,NaN,NaN,NaN,...,913904912,JUDICIAL VOLUNTARIA,NaN,50.0,https://subastas.boe.es/./detalleSubasta.php?i...,NaN,2500.0,2500.0,Si (Las visitas se podran realizar de lunes a ...,NaN
3,BOE-B-2019-35282,NaN,0.0,No constan,28079060HM,NaN,administracionbienes.orga@mjusticia.es,NaN,NaN,NaN,...,913904912,JUDICIAL VOLUNTARIA,NaN,50.0,https://subastas.boe.es/./detalleSubasta.php?i...,NaN,2500.0,2500.0,Si (Las visitas se podran realizar de lunes a ...,NaN
4,BOE-B-2019-35282,NaN,0.0,No constan,28079060HM,NaN,administracionbienes.orga@mjusticia.es,NaN,NaN,NaN,...,913904912,JUDICIAL VOLUNTARIA,NaN,50.0,https://subastas.boe.es/./detalleSubasta.php?i...,NaN,2500.0,2500.0,Si (Las visitas se podran realizar de lunes a ...,NaN


In [33]:
new=pd.DataFrame(df.DIRECCION.str.split(' ', expand=True))
calle = ['CALLE', 'C/', 'CL', 'c/', 'calle']
plaza = ['PLAZA', 'PZ/', 'Pz/', 'Pz.', 'PZ.']
paseo = ['PASEO', 'PS', 'paseo']
avenida = ['AVENIDA', 'AVDA', 'AV', 'AV.', 'Avenida.', 'Avda.']
for c in calle:
    new[0] = new[0].str.replace(c, 'Calle')
for c in plaza:
    new[0] = new[0].str.replace(c, 'Plaza')
for c in paseo:
    new[0] = new[0].str.replace(c, 'Paseo')
for c in avenida:
    new[0] = new[0].str.replace(c, 'Avenida')
print(new[0].value_counts())

Calle             11428
Plaza              7670
Avenida            2996
Paseo              2221
PINTOR              362
Solar               296
Po                  191
CARRERA             136
PARQUE              135
FINCA               127
SAU                 114
CARRETERA           107
VELAZQUEZ           101
CAMINO              101
GUARDA              101
Otros               100
SOLAR                95
PARCELA              88
PARAJE               82
GRAN                 80
CalleSAN             72
TERRENO              66
PZ                   61
UE1                  61
LOCAL                60
Sonrisa,             58
ED                   58
PISO                 52
Vivienda             52
RUA                  48
                  ...  
CalleClavel           1
Monteleon             1
Callejon              1
CallePORTUGAL         1
TVA.                  1
LICENCIA              1
VENTORRO              1
PICOS                 1
caale                 1
BLANCA                1
CalleSANTA      

In [54]:
n = new[0]
for i in range(1,51):
    n = n.str.cat(pd.DataFrame(new[i]),sep=' ')
print(n.head())

new[0].str.cat(new[1],sep=' ')

0    NaN
1    NaN
2    NaN
3    NaN
4    NaN
Name: 0, dtype: object


0                Calle SAN
1                Calle SAN
2                Calle SAN
3                Calle SAN
4                Calle SAN
5                Calle SAN
6                No consta
7                 Paseo de
8                    PZ DE
9                Paseo DEL
10               Paseo DEL
11       PARAJE CORTIJILLO
12       PARAJE CORTIJILLO
13       PARAJE CORTIJILLO
14       PARAJE CORTIJILLO
15       PARAJE CORTIJILLO
16       PARAJE CORTIJILLO
17       PARAJE CORTIJILLO
18       PARAJE CORTIJILLO
19       PARAJE CORTIJILLO
20       PARAJE CORTIJILLO
21       PARAJE CORTIJILLO
22       PARAJE CORTIJILLO
23       PARAJE CORTIJILLO
24       PARAJE CORTIJILLO
25       PARAJE CORTIJILLO
26       PARAJE CORTIJILLO
27       PARAJE CORTIJILLO
28       PARAJE CORTIJILLO
29       PARAJE CORTIJILLO
               ...        
30289           Calle VEGA
30290           Calle VEGA
30291           Calle VEGA
30292           Calle VEGA
30293           Calle VEGA
30294           Calle VEGA
3